# Introdução

## 📘 Sobre este Estudo

Este notebook foi elaborado com fins **didáticos e demonstrativos** para mostrar como aplicar técnicas de **inferência estatística**, utilizando tanto métodos **clássicos (paramétricos)** quanto **computacionais** (como bootstrap e testes de aleatorização).

---

## 🛒 Contexto do Problema

A empresa fictícia **RetailX**, uma varejista digital, lançou uma campanha para testar o impacto de diferentes estratégias promocionais sobre o comportamento de seus clientes. A empresa está especialmente interessada em entender:

- Se as campanhas aumentam o engajamento no site
- Se as campanhas influenciam a taxa de conversão (compra)
- Se há impacto no valor total gasto

Para isso, um experimento foi conduzido com **quatro grupos de clientes**, atribuídos aleatoriamente.

---

## 🎯 Objetivo do Estudo

Avaliar se **diferentes tipos de campanha** geram **diferenças estatisticamente significativas** em:
- Taxa de conversão (realizou compra ou não)
- Valor de compra (gasto em reais)
- Número de visitas ao site (antes e depois da campanha)
---

## 🧾 Descrição das Variáveis

| Variável            | Tipo        | Descrição                                                                 |
|---------------------|-------------|---------------------------------------------------------------------------|
| `grupo`             | Categórica  | **Campanha recebida pelo cliente**:<br>• `A`: grupo **controle** (não recebeu estímulo)<br>• `B`: cupom de desconto<br>• `C`: frete grátis<br>• `D`: cashback |
| `evento`            | Binária     | Indicador de conversão: o cliente **realizou uma compra?** (`1` = sim, `0` = não) |
| `score`             | Contínua    | Valor total gasto pelo cliente (em reais) durante o período da campanha  |
| `pre_treinamento`   | Contínua    | Número de visitas ao site **antes da campanha**                           |
| `pos_treinamento`   | Contínua    | Número de visitas ao site **após a campanha**                             |

---

## 🧠 O que significa “grupo controle”?

O grupo **controle** (`grupo = A`) é formado por clientes que **não receberam nenhuma campanha promocional**. Esse grupo serve como **referência neutra**, permitindo que possamos comparar com os demais grupos e **isolar o efeito da campanha**.

> Exemplo: Se o grupo B (cupom) tiver maior conversão do que o grupo A (controle), é possível afirmar com maior confiança que o cupom teve impacto real — desde que a diferença seja estatisticamente significativa.

# Análises

In [1]:
# SETUP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, t, f, chi2

In [2]:
# LOAD: A base já está sanitizada.
base = pd.read_excel("dados/base_retailx.xlsx")
base.head()

,grupo,evento,score,pre_treinamento,pos_treinamento
0,A,0,0.00,6,6
1,A,1,70.68,4,4
2,A,1,60.23,6,6
3,A,1,102.16,8,8
4,A,0,0.00,1,2


## 1. Construa um intervalo de confiança de 95% para estimar a proporção de clientes que realizaram compra.

In [3]:
# Variável observada
X = base["evento"].values

# Tamanho da amostra
n = X.shape[0]

# Proporção amostral observada
p_obs = X.mean()

print(f"n = {n}")
print(f"p_obs = {np.round(p_obs, 2)}")

n = 300
p_obs = 0.57


### Modelo Paramétrico

<img src="img/e1a.jpeg" width="500">
<img src="img/e1b.jpeg" width="500">

In [4]:
# Melhor estimativa para p
p = p_obs

# Validação de aproximação pela normal
print((n*p >= 10) & (n*(1-p) >= 10))

True


In [5]:
# Erro Padrão
SE = np.sqrt(p*(1-p)/n)
print(f"SE = {np.round(SE, 2)}")

SE = 0.03


In [6]:
# Obtenção do Zc
gama = .95
q    = (1- gama)/2 + gama
Zc   = norm.ppf(q=q)
print(f"Zc = {np.round(Zc, 2)}")

Zc = 1.96


In [7]:
# Intervalo de confiança
p0 = p_obs - Zc*SE
p1 = p_obs + Zc*SE
print(f"IC = [{np.round(p0, 2)}, {np.round(p1, 2)}]")
print(f"IC = {np.round(p_obs, 2)} +- {np.round(Zc, 2)} * {np.round(SE, 2)}")

IC = [0.51, 0.62]
IC = 0.57 +- 1.96 * 0.03


**Conclusão**

- Temos **95% de confiança** de que o verdadeiro valor da proporção populacional \(p\) está entre **51% e 62%**. Em outras palavras, acreditamos com 95% de confiança que a **taxa de conversão dos clientes dessa loja** se encontra nesse intervalo.

- Essa confiança **não deve ser interpretada** como a probabilidade de que este intervalo específico contenha \(p\), pois o valor de \(p\) é fixo (embora desconhecido), e o intervalo é aleatório.

- A interpretação correta é: Se repetíssemos esse mesmo processo de amostragem e construção de intervalo muitas vezes, em **95% dos casos** os intervalos obtidos conteriam o verdadeiro valor de \(p\).

- Portanto, com base nessa confiança no processo, podemos selecionar **este único intervalo** e afirmar com **95% de confiança** que o verdadeiro valor de \(p\) está entre **51% e 62%**.


## 2. Podemos afirmar que a taxa de conversão dos clientes é superior que 50%?

**Hipóteses do Teste**

- **H₀ (Hipótese nula):** A taxa de conversão na população **não é superior a 50%**. O valor observado pode ser explicado apenas pela variabilidade amostral (acaso na seleção da amostra).
- **H₁ (Hipótese alternativa):** A taxa de conversão na população **é superior a 50%**.

**Notação formal:**

- H₀: p = po | (po = 50%)
- H₁: p > po



### Modelo Paramétrico

<img src="img/e2.jpeg" width="500">

In [8]:
# Nível de significância do teste
alpha = .05
print(f"alpha = {np.round(alpha, 2)}")

alpha = 0.05


In [9]:
# Distribuição sob H0
po = .5
print(f"p0 = {np.round(po, 2)}")

p0 = 0.5


In [10]:
# Validação de aproximação pela normal
print((n*p0 >= 10) & (n*(1-p0) >= 10))

True


In [11]:
# Padronizando o valor observado
Z_obs = (p_obs - po)/np.sqrt(po*(1-po)/n)
print(f"Z_obs = {np.round(Z_obs, 2)}")

Z_obs = 2.31


In [12]:
# Calculando p_value (probabilidade do acaso gerar um valor tão extremo quanto o observado)
p_value = 1 - norm.cdf(Z_obs)

# Decisão
if p_value < alpha:
    print(f"p-valor = {p_value:.2f} < α = {alpha} → Rejeitamos H0")
else:
    print(f"p-valor = {p_value:.2f} ≥ α = {alpha} → Não rejeitamos H0")


p-valor = 0.01 < α = 0.05 → Rejeitamos H0


**Conclusão**

- Ao nível de significância de 5% (probabilidade de rejeitar incorretamente a hipótese nula), temos evidência suficiente para rejeitar H₀ em favor de H₁.  
Com isso, concluímos que a taxa de conversão dos clientes é **superior a 50%**.


## 3. Construa um intervalo de confiança de 95% para estimar a diferença na taxa de conversão entre os grupos B e D.

In [13]:
# Grupo B: Teve cupom de desconto

# Variável observada
XB = base.loc[(base["grupo"] == "B"), "evento"].values

# Tamanho da amostra
nB = XB.shape[0]

# Proporção amostral observada
pB_obs = XB.mean()

print(f"nB = {nB}")
print(f"pB_obs = {np.round(pB_obs, 2)}")

nB = 75
pB_obs = 0.63


In [14]:
# Grupo D: Teve cashback

# Variável observada
XD = base.loc[(base["grupo"] == "D"), "evento"].values

# Tamanho da amostra
nD = XD.shape[0]

# Proporção amostral observada
pD_obs = XD.mean()

print(f"nD = {nD}")
print(f"pD_obs = {np.round(pD_obs, 2)}")

nD = 75
pD_obs = 0.61


In [15]:
# Diferença observada
diff_B_D_obs = pB_obs - pD_obs
print(f"diff_B_D_obs = {np.round(diff_B_D_obs, 2)}")

diff_B_D_obs = 0.01


### Modelo Paramétrico

<img src="img/e3a.jpeg" width="500">

<img src="img/e3b.jpeg" width="500">

In [16]:
# Melhor estimativa para pB
pB = pB_obs

# Validação de aproximação pela normal
print((nB*pB >= 10) & (nB*(1-pB) >= 10))

True


In [17]:
# Melhor estimativa para pD
pD = pD_obs

# Validação de aproximação pela normal
print((nD*pD >= 10) & (nD*(1-pD) >= 10))

True


In [18]:
# Erro Padrão
SE = np.sqrt(pB*(1-pB)/nB + pD*(1-pD)/nD)
print(f"SE = {np.round(SE, 2)}")

SE = 0.08


In [19]:
# Obtenção do Zc
gama = .95
q    = (1- gama)/2 + gama
Zc   = norm.ppf(q=q)
print(f"Zc = {np.round(Zc, 2)}")

Zc = 1.96


In [20]:
# Intervalo de confiança
d0 = diff_B_D_obs - Zc*SE
d1 = diff_B_D_obs + Zc*SE
print(f"IC = [{np.round(d0, 2)}, {np.round(d1, 2)}]")
print(f"IC = {np.round(diff_B_D_obs, 2)} +- {np.round(Zc, 2)} * {np.round(SE, 2)}")

IC = [-0.14, 0.17]
IC = 0.01 +- 1.96 * 0.08


**Conclusão**

- Com **95% de confiança**, estimamos que a diferença populacional nas taxas de conversão entre os grupos B e D (pB - pD) esteja entre **-14 p.p. e 17 p.p.**
- Em outras palavras, acreditamos que a taxa de conversão do grupo B pode ser até 14 pontos percentuais **menor** ou até 17 pontos percentuais **maior** do que a do grupo D.

## 4. Podemos concluir que a taxa de conversão nos grupos B e D são diferentes?

O resultado apresentado no item 3 já responde à pergunta: com **95% de confiança**, estimamos que a diferença populacional nas taxas de conversão entre os grupos B e D (pB - pD) esteja entre **-14 p.p. e 17 p.p.**

Como esse intervalo inclui o valor **0**, **não temos evidência estatística suficiente** para concluir que existe uma diferença significativa entre as taxas de conversão dos dois grupos, ao nível de significância de 5%.

Além disso, a realização de um **teste de hipótese** formal para a diferença entre proporções levaria à **mesma conclusão**, reforçando que não há evidência de diferença estatisticamente significativa — o que faremos na sequência.

**Hipóteses do Teste**

- **H₀ (Hipótese nula):** Não há diferença na taxa de conversão entre os grupos B e D. A diferença observada pode ser atribuída à variabilidade amostral (isto é, ao acaso na seleção da amostra).
- **H₁ (Hipótese alternativa):** Existe uma diferença real entre as taxas de conversão dos grupos B e D.

**Notação formal:**

- H₀: pB - pD = do | (do = 0)
- H₁: pB - pD != do



### Modelo Paramétrico

<img src="img/e4.jpeg" width="500">

In [21]:
# Nível de significância do teste
alpha = .05
print(f"alpha = {np.round(alpha, 2)}")

alpha = 0.05


In [22]:
# Distribuição sob H0
po = (nB * pB_obs + nD * pD_obs) / (nB + nD)
print(f"p0 = {np.round(po, 2)}")

p0 = 0.62


In [23]:
# Validação de aproximação pela normal
print((nB*p0 >= 10) & (nB*(1-p0) >= 10))
print((nD*p0 >= 10) & (nD*(1-p0) >= 10))

True
True


In [24]:
# Padronizando o valor observado
Z_obs = (pB_obs - pD_obs)/np.sqrt((po*(1-po)/nB) + (po*(1-po)/nD)) 
print(f"Z_obs = {np.round(Z_obs, 2)}")

Z_obs = 0.17


In [25]:
# Calculando p_value (probabilidade do acaso gerar um valor tão extremo quanto o observado)
p_value = norm.cdf(-Z_obs) + (1 - norm.cdf(Z_obs))

# Decisão
if p_value < alpha:
    print(f"p-valor = {p_value:.2f} < α = {alpha} → Rejeitamos H0")
else:
    print(f"p-valor = {p_value:.2f} ≥ α = {alpha} → Não rejeitamos H0")

p-valor = 0.87 ≥ α = 0.05 → Não rejeitamos H0


**Conclusão**

- Ao nível de significância de 5%, não encontramos evidência estatística suficiente para rejeitar H₀ em favor de H₁.  
Com isso, não podemos concluir que exista diferença significativa nas taxas de conversão entre os grupos B e D.

## 5. A taxa de conversão (`evento`) está associada ao tipo de campanha (`grupo`)?

### Modelo Paramétrico

<img src="img/e5a.jpeg" width="500">
<img src="img/e5b.jpeg" width="500">

In [26]:
# Resultado Observado
X = base["evento"].values
Y = base["grupo"].values

obs = pd.crosstab(Y, X)
obs

col_0,0,1
row_0,,
A,40,35
B,28,47
C,33,42
D,29,46


In [27]:
# Resultado Esperado se X e Y não possuem relação

# Fração 0's e 1's geral
frac = (obs.sum()/obs.sum().sum()).values.reshape(1,2)

# Soma por linha
sum_by_row = obs.sum(axis=1).values.reshape(4,1)

# Esperado
exp = pd.DataFrame(np.matmul(sum_by_row, frac))
exp

,0,1
0,32.5,42.5
1,32.5,42.5
2,32.5,42.5
3,32.5,42.5


In [28]:
# X2 observado
obs = obs.values 
exp = exp.values
X2_obs = (((obs - exp)**2)/exp).sum()
print(f"X2_obs = {np.round(X2_obs, 2)}")

X2_obs = 4.83


**Hipóteses do Teste (Qui-quadrado de independência)**

- **H₀ (Hipótese nula):** Não há associação entre a taxa de conversão e o tipo de campanha. A diferença observada pode ser explicada apenas pela variabilidade amostral (isto é, pelo acaso).
- **H₁ (Hipótese alternativa):** Existe uma associação entre a taxa de conversão e o tipo de campanha.


In [29]:
# Construção da distribuição sob H0: Como as premissas abaixo são satisfeitas, podemos avaliar qual a probabilidade do acaso 
# produzir um valor tão extremo quanto X2_obs em uma distribuição Qui Quadrado (variáveis sem relação)
# a. As observações são independentes
# b. Supondo que não haja relação entre as variáveis
# c. Pelo menos 80% das células da tabela esperada possuem valores >= 5
# d. Nenhuma célula com valor esperado < 1

# Graus de liberdade
r = obs.shape[0]
c = obs.shape[1]
df = (r - 1) * (c - 1)
print(f"df = {np.round(df, 2)}")

# Calculando p_value (probabilidade do acaso gerar um valor tão extremo quanto o observado)
# P(X2 >= X2_obs)
p_value = 1 - chi2.cdf(X2_obs, df)

# Decisão
if p_value < alpha:
    print(f"p-valor = {p_value:.2f} < α = {alpha} → Rejeitamos H0")
else:
    print(f"p-valor = {p_value:.2f} ≥ α = {alpha} → Não rejeitamos H0")

df = 3
p-valor = 0.18 ≥ α = 0.05 → Não rejeitamos H0


**Conclusão**

- Ao nível de significância de 5%, **não encontramos evidência estatística suficiente para rejeitar H₀ em favor de H₁**.  
Dessa forma, **não há evidência de associação entre a ocorrência do evento (ex: conversão) e o tipo de campanha aplicada**.


## 6. Construa um intervalo de confiança de 95% para estimar o valor médio de compra (`score`) de todos os clientes.

In [30]:
# Variável observada: Somente clientes que realizaram alguma compra
X = base.loc[base["evento"] == 1, "score"].values

# Tamanho da amostra
n = X.shape[0]

# Média amostral observada
x_obs = X.mean()

# Desvio padrão amostral
s = X.std(ddof=1)

print(f"n = {n}")
print(f"x_obs = {np.round(x_obs, 2)}")
print(f"s = {np.round(s, 2)}")

n = 170
x_obs = 77.16
s = 13.52


### Modelo Paramétrico

<img src="img/e6a.jpeg" width="500">
<img src="img/e6b.jpeg" width="500">

In [31]:
# Quando o tamanho da amostra é grande, o Teorema Central do Limite permite aproximar a distribuição da média amostral por uma normal.
# No entanto, como a variância populacional é desconhecida, usamos a variância amostral como estimativa.
# Isso introduz variabilidade adicional na estatística de teste, razão pela qual usamos a distribuição t de Student em vez da normal padrão.

# Erro Padrão
SE = np.sqrt((s**2)/n)
print(f"SE = {np.round(SE, 2)}")

# Obtenção do tc
gama = .95
q    = (1- gama)/2 + gama
df   = n -1
tc   = t.ppf(df = df, q=q)
print(f"tc = {np.round(tc, 2)}")

# Intervalo de confiança
x0 = x_obs - tc*SE
x1 = x_obs + tc*SE
print(f"IC = [{np.round(x0, 2)}, {np.round(x1, 2)}]")
print(f"IC = {np.round(x_obs, 2)} +- {np.round(tc, 2)} * {np.round(SE, 2)}")

SE = 1.04
tc = 1.97
IC = [75.11, 79.21]
IC = 77.16 +- 1.97 * 1.04


**Conclusão**

- Temos **95% de confiança** de que o verdadeiro valor do score populacional u está entre **75,11 e 79,21**.  
Em outras palavras, acreditamos que o **valor médio de compra da população** se encontra nesse intervalo com 95% de confiança.


## 7. O valor médio de compra (`score`) de todos os clientes é maior que R$ 70?

**Hipóteses do Teste**

- **H₀ (Hipótese nula):** O valor médio de compra **não é maior que 70 reais**. A média observada pode ser explicada apenas pela variabilidade amostral (isto é, pelo acaso na seleção da amostra).
- **H₁ (Hipótese alternativa):** O valor médio de compra **é maior que 70 reais**.

**Notação formal:**

- H₀: u = uo | (uo = 70)
- H₁: u > uo



### Modelo Paramétrico

<img src="img/e7.jpeg" width="500">

In [32]:
# Nível de significância do teste
alpha = .05
print(f"alpha = {np.round(alpha, 2)}")

alpha = 0.05


In [33]:
# Distribuição sob H0
# - Supondo uma população com uo = 70
# - Com variância estimada igual a variancia amostral

# Quando o tamanho da amostra é grande, o Teorema Central do Limite permite aproximar a distribuição da média amostral por uma normal.
# No entanto, como a variância populacional é desconhecida, usamos a variância amostral como estimativa.
# Isso introduz variabilidade adicional na estatística de teste, razão pela qual usamos a distribuição t de Student em vez da normal padrão.

# u0
uo = 70
print(f"uo = {np.round(uo, 2)}")

# Erro Padrão
SE = np.sqrt((s**2)/n)
print(f"SE = {np.round(SE, 2)}")

# Obtenção do t_obs
t_obs = (x_obs - uo)/SE
print(f"t_obs = {np.round(t_obs, 2)}")

# Calculando p_value (probabilidade do acaso gerar um valor tão extremo quanto o observado)
# P(t >= t_obs)
p_value = 1 - t.cdf(t_obs, df)

# Decisão
if p_value < alpha:
    print(f"p-valor = {p_value:.2f} < α = {alpha} → Rejeitamos H0")
else:
    print(f"p-valor = {p_value:.2f} ≥ α = {alpha} → Não rejeitamos H0")

uo = 70
SE = 1.04
t_obs = 6.9
p-valor = 0.00 < α = 0.05 → Rejeitamos H0


**Conclusão**

- Ao nível de significância de 5% — ou seja, assumindo um risco de 5% de rejeitar incorretamente a hipótese nula —, encontramos **evidência estatística suficiente para rejeitar H₀ em favor de H₁**.  
Dessa forma, **concluímos que o valor médio de compra é superior a 70 reais**.

## 8. Construa um intervalo de confiança de 95% para estimar a diferença no valor médio de compra entre os grupos B e D.

### Modelo Paramétrico

<img src="img/e8a.jpeg" width="500">
<img src="img/e8b.jpeg" width="500">

In [34]:
# Grupo B: Teve cupom de desconto

# Variável observada: Somente clientes que realizaram alguma compra
XB = base.loc[(base["evento"] == 1) & (base["grupo"] == "B"), "score"].values

# Tamanho da amostra
nB = XB.shape[0]

# Média amostral observada
xB_obs = XB.mean()

# Desvio padrão amostral
sB = XB.std(ddof=1)

print(f"nB = {nB}")
print(f"xB_obs = {np.round(xB_obs, 2)}")
print(f"sB = {np.round(sB, 2)}")

nB = 47
xB_obs = 76.52
sB = 11.93


In [35]:
# Grupo D: Teve cupom de desconto

# Variável observada: Somente clientes que realizaram alguma compra
XD = base.loc[(base["evento"] == 1) & (base["grupo"] == "D"), "score"].values

# Tamanho da amostra
nD = XD.shape[0]

# Média amostral observada
xD_obs = XD.mean()

# Desvio padrão amostral
sD = XD.std(ddof=1)

print(f"nD = {nD}")
print(f"xD_obs = {np.round(pD_obs, 2)}")
print(f"sD = {np.round(sD, 2)}")

nD = 46
xD_obs = 0.61
sD = 11.28


In [36]:
# Diferença observada
diff_B_D_obs = xB_obs - xD_obs
print(f"diff_B_D_obs = {np.round(diff_B_D_obs, 2)}")

diff_B_D_obs = -6.78


In [37]:
# Quando o tamanho da amostra é grande, o Teorema Central do Limite permite aproximar a distribuição da média amostral por uma normal.
# No entanto, como a variância populacional é desconhecida, usamos a variância amostral como estimativa.
# Isso introduz variabilidade adicional na estatística de teste, razão pela qual usamos a distribuição t de Student em vez da normal padrão.

# Erro Padrão
SE = np.sqrt((sB**2)/nB + (sD**2)/nD)
print(f"SE = {np.round(SE, 2)}")

# Obtenção do tc
gama = .95
q    = (1- gama)/2 + gama
df   = (((sB**2)/nB + (sD**2)/nD)**2)/((((sB**2)/nB)**2)/(nB - 1) + (((sD**2)/nD)**2)/(nD - 1))
tc   = t.ppf(df = df, q=q)
print(f"tc = {np.round(tc, 2)}")

# Intervalo de confiança
d0 = diff_B_D_obs - tc*SE
d1 = diff_B_D_obs + tc*SE
print(f"IC = [{np.round(d0, 2)}, {np.round(d1, 2)}]")
print(f"IC = {np.round(diff_B_D_obs, 2)} +- {np.round(tc, 2)} * {np.round(SE, 2)}")

SE = 2.41
tc = 1.99
IC = [-11.56, -2.0]
IC = -6.78 +- 1.99 * 2.41


**Conclusão**

- Com **95% de confiança**, estimamos que a **diferença média no valor de compra entre os grupos B e D** está entre **−11,56 e −2,00**.
- Em outras palavras, acreditamos que os clientes do grupo B gastam, em média, entre **2 e 11,56 unidades a menos** do que os do grupo D.


## 9. Podemos afirmar que o grupo D gerou um valor médio de compra diferente do que o grupo B?

**Hipóteses do Teste**

- **H₀ (Hipótese nula):** Não há diferença no valor médio de compra entre os grupos B e D. Qualquer diferença observada pode ser explicada apenas pela variabilidade amostral (isto é, pelo acaso na seleção da amostra).
- **H₁ (Hipótese alternativa):** Há diferença no valor médio de compra entre os grupos B e D.

**Notação formal:**

- H₀: uB - uD = do | (do = 0)
- H₁: uB - uD != do

### Modelo Paramétrico

<img src="img/e9.jpeg" width="500">

In [38]:
# Nível de significância do teste
alpha = .05
print(f"alpha = {np.round(alpha, 2)}")

alpha = 0.05


In [39]:
# Distribuição sob H0
# - Supondo uma população com uB - uD = do = 0
# - Com variância estimada sB**2/nB + sD**2/nD

# Quando o tamanho da amostra é grande, o Teorema Central do Limite permite aproximar a distribuição da diferença da média amostral por uma normal.
# No entanto, como a variância populacional é desconhecida, usamos a variância amostral como estimativa.
# Isso introduz variabilidade adicional na estatística de teste, razão pela qual usamos a distribuição t de Student em vez da normal padrão.

# do
do = 0
print(f"do = {np.round(do, 2)}")

# Erro Padrão
SE = np.sqrt((sB**2)/nB + (sD**2)/nD)
print(f"SE = {np.round(SE, 2)}")

# Obtenção do t_obs
t_obs = diff_B_D_obs/SE
print(f"t_obs = {np.round(t_obs, 2)}")

# Calculando p_value (probabilidade do acaso gerar um valor tão extremo quanto o observado)
# P(t >= t_obs)
df      = (((sB**2)/nB + (sD**2)/nD)**2)/((((sB**2)/nB)**2)/(nB - 1) + (((sD**2)/nD)**2)/(nD - 1))
p_value = t.cdf(t_obs, df)*2

# Decisão
if p_value < alpha:
    print(f"p-valor = {p_value:.2f} < α = {alpha} → Rejeitamos H0")
else:
    print(f"p-valor = {p_value:.2f} ≥ α = {alpha} → Não rejeitamos H0")

do = 0
SE = 2.41
t_obs = -2.82
p-valor = 0.01 < α = 0.05 → Rejeitamos H0


**Conclusão**

- Ao nível de significância de 5% — ou seja, assumindo um risco de 5% de rejeitar incorretamente a hipótese nula —, encontramos **evidência estatística suficiente para rejeitar H₀ em favor de H₁**.  
- Dessa forma, **concluímos que há diferença significativa no valor médio de compra entre os grupos B e D**.